In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import seaborn as sns
sns.set()

# Import data and remove useless columns 

## Import data

In [ ]:
employement_rate = pd.read_csv('./data/labour_market/EMP_RATE_by_birth_place_and_EDUC.csv')
min_wage = pd.read_csv('./data/labour_market/real minimum wage from 2001 to 2019.csv')
avg_duration_unemployment = pd.read_csv('./data/labour_market/average duration of unemployment.csv')
strictness_emply_protec = pd.read_csv('./data/labour_market/strictness of employment protection from 1998 to 2018.csv')
short_time_workers = pd.read_csv('./data/labour_market/percentage of short time workers from 2000.csv')
involuntary_pt_workers = pd.read_csv('./data/labour_market/percentage of involuntary part time workers from 2000.csv')
well_matched_skills_job = pd.read_csv('./data/labour_market/percentage of well matched skills for jobs.csv', dtype='unicode')
ft_and_pt_employ = pd.read_csv('./data/labour_market/percentage of full time part time employment.csv')
marginally_attached_workers = pd.read_csv('./data/labour_market/percentage of marginally attached workers from 2000.csv')

## Dictionary with the country code of all OECD countries

In [2]:
oecd_countries = {'AUS': 'Australia', 'AUT': 'Austria', 'BEL': 'Belgium', 'CAN': 'Canada', 'CHL': 'Chile', 'COL': 'Colombia', 'CZE': 'Czech Republic', 'DNK': 'Denmark', 'EST': 'Estonia', 'FIN': 'Finland', 'FRA': 'France', 'DEU': 'Germany', 'GRC': 'Greece', 'HUN': 'Hungary', 'ISL': 'Iceland', 'IRL': 'Ireland', 'ISR': 'Israel', 'ITA': 'Italy', 'JPN': 'Japan', 'KOR': 'Korea', 'LVA': 'Latvia', 'LTU': 'Lithuania', 'LUX': 'Luxembourg', 'MEX': 'Mexico', 'NLD': 'Netherlands', 'NZL': 'New Zealand', 'NOR': 'Norway', 'POL': 'Poland', 'PRT': 'Portugal', 'SVK': 'Slovakia', 'SVN': 'Slovenia', 'ESP': 'Spain', 'SWE': 'Sweden', 'CHE': 'Switzerland', 'TUR': 'Turkey', 'GBR': 'United Kingdom', 'USA': 'United States'}

## Preparation of the data sets

### Removal of unnecessary columns and countries

### Sorting of data by date